In [ ]:
# IMPORT ALL YOUR LIBRARIES
!pip install transformers
!pip install datasets
!pip install accelerate >=0.21.0
!pip install transformers[torch]

import torch
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

In [ ]:
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/sun.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/som.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/deu.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/deu_c.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/dev/rus_c.csv"

File ‘/kaggle/working/deu.csv’ already there; not retrieving.

File ‘/kaggle/working/rus_c.csv’ already there; not retrieving.



In [ ]:
df = pd.read_csv('/kaggle/working/deu.csv')
df.head()

id                                               text  \
0  deu_train_track_a_00001  Nein nein nix da, fuck den schön weiter ab bis...   
1  deu_train_track_a_00002  Vor 100 Jahren ging man auf die Strasse weil m...   
2  deu_train_track_a_00003                Wann endlich Gabby Epstein heiraten   
3  deu_train_track_a_00004  Wegen dem Song 1001 Nacht, dem einen Phantomto...   
4  deu_train_track_a_00005  Dann sollten wir versuchen, sie zu erreichen. ...   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0      1        0     0    0        0         0  
1      1        1     0    0        0         0  
2      0        0     0    0        0         0  
3      0        0     0    1        0         0  
4      0        0     0    0        0         0

In [ ]:
df_test = pd.read_csv('/kaggle/working/rus_c.csv')
df_test.head()

id                                               text  \
0  rus_dev_track_c_00001                           как всегда хуйни зададут   
1  rus_dev_track_c_00002  Никто не знает силы своих способностей, пока о...   
2  rus_dev_track_c_00003                           Ого!!) 62 с первого раза   
3  rus_dev_track_c_00004  Блин! Могу сидеть только в твиттере, а так хоч...   
4  rus_dev_track_c_00005           Даа! Я это сделала!! Мама уезжает!))))))   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    NaN      NaN   NaN  NaN      NaN       NaN  
4    NaN      NaN   NaN  NaN      NaN       NaN

In [ ]:
# from bs4 import BeautifulSoup
# import re

# # # Step 2: Clean the text data
# # def clean_text(text):
# #     # Remove HTML tags
# #     text = BeautifulSoup(text, "html.parser").get_text()
# #     # Remove numbers (but keep special characters)
# #     text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
# #     # Convert to lowercase
# #     text = text.lower()
# #     # Remove extra whitespace
# #     text = ' '.join(text.split())
# #     return text

# # # Apply the clean_text function
# # df_test['text'] = df_test['text'].apply(clean_text)

# # Step 3-5: (Same as your original code)
# df_test = df_test[df_test['text'].notna() & (df_test['text'] != '')]

# # Ensure emotion columns are integers
# # emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['Anger', 'Disgust','Fear', 'Joy', 'Sadness', 'Surprise']
# df_test[emotion_columns] = df_test[emotion_columns]

# # Save the processed data back to CSV
# output_path = '/kaggle/working/cleaned_test_english.csv'
# df_test.to_csv(output_path, index=False)

# print(f"Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.")

# Display statistics and processed examples
print(f"Total samples: {len(df_test)}")
print("\nEmotion distribution in the dataset:")
print(df_test[emotion_columns].sum())
print("\nFirst few processed examples:")
print(df_test.head())


Total samples: 401

Emotion distribution in the dataset:
Anger       0.0
Disgust     0.0
Fear        0.0
Joy         0.0
Sadness     0.0
Surprise    0.0
dtype: float64

First few processed examples:
                      id                                               text  \
0  rus_dev_track_c_00001                           как всегда хуйни зададут   
1  rus_dev_track_c_00002  Никто не знает силы своих способностей, пока о...   
2  rus_dev_track_c_00003                           Ого!!) 62 с первого раза   
3  rus_dev_track_c_00004  Блин! Могу сидеть только в твиттере, а так хоч...   
4  rus_dev_track_c_00005           Даа! Я это сделала!! Мама уезжает!))))))   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    NaN      NaN   NaN  NaN      NaN       NaN  
4    NaN      NaN   NaN  NaN      NaN       NaN  


In [ ]:
if 'id' in df.columns:
    df = df.drop(['id'], axis=1)
from bs4 import BeautifulSoup
import re

# # Step 2: Clean the text data
# def clean_text(text):
#     # Remove HTML tags
#     text = BeautifulSoup(text, "html.parser").get_text()
#     # Remove numbers (but keep special characters)
#     text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
#     # Convert to lowercase
#     text = text.lower()
#     # Remove extra whitespace
#     text = ' '.join(text.split())
#     return text

# # df['text'] = df['text'].apply(clean_text)

# Step 3: Remove rows with empty text
df = df[df['text'].notna() & (df['text'] != '')]

# Step 4: Ensure emotion columns are integers
emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# emotion_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
df[emotion_columns] = df[emotion_columns].astype(int)

# Step 5: Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_deu.csv'
df.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_deu.csv' has been created in the /kaggle/working directory.")

# Optional: Display some statistics about the data
print(f"Total samples: {len(df)}")
print("\nEmotion distribution in the dataset:")
print(df[emotion_columns].sum())

# Optional: Display a few processed examples
print("\nFirst few processed examples:")
print(df.head())

Data preprocessing completed. File 'cleaned_deu.csv' has been created in the /kaggle/working directory.
Total samples: 2603

Emotion distribution in the dataset:
Anger       768
Disgust     832
Fear        239
Joy         541
Sadness     516
Surprise    159
dtype: int64

First few processed examples:
                                                text  Anger  Disgust  Fear  \
0  Nein nein nix da, fuck den schön weiter ab bis...      1        0     0   
1  Vor 100 Jahren ging man auf die Strasse weil m...      1        1     0   
2                Wann endlich Gabby Epstein heiraten      0        0     0   
3  Wegen dem Song 1001 Nacht, dem einen Phantomto...      0        0     0   
4  Dann sollten wir versuchen, sie zu erreichen. ...      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        0         0  
2    0        0         0  
3    1        0         0  
4    0        0         0  


## German to English

In [ ]:
import pandas as pd
from transformers import MarianTokenizer, MarianMTModel

In [ ]:
from transformers import pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en", device=0)    # Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
import pandas as pd
from transformers import pipeline

# Load the CSV file
input_file = "/kaggle/working/deu.csv"  # Replace with your input file path
output_file = "/kaggle/working/deu-to-eng.csv"  # Replace with your desired output file path

# Load the translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en", device=0)

# Function to translate text
def translate_text(text, translator):
    try:
        # Translate using the pipeline
        translation = translator(text, truncation=True)
        return translation[0]["translation_text"]  # Extract translated text
    except Exception as e:
        print(f"Error translating text: {text}\n{e}")
        return text  # Return original text if translation fails

# Load the dataset
df = pd.read_csv(input_file)

# Check if 'text' column exists
if "text" not in df.columns:
    raise ValueError("The CSV file does not contain a 'text' column.")

# Translate the 'text' column
df["text"] = df["text"].apply(lambda x: translate_text(x, translator))

# Save the updated dataframe to a new CSV
df.to_csv(output_file, index=False)

print(f"Translation completed. Translated file saved at: {output_file}")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Translation completed. Translated file saved at: /kaggle/working/deu-to-eng.csv


In [ ]:
df.to_csv('deu-to-eng.csv', index=False)

## Russian to English

In [ ]:
from transformers import MarianTokenizer, MarianMTModel

model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from transformers import pipeline

# Load the CSV file
input_file = "/kaggle/working/rus_c.csv"  # Replace with your input file path
output_file = "/kaggle/working/rus-to-eng.csv"  # Replace with your desired output file path

# Load the translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ru-en", device=0)

# Function to translate text
def translate_text(text, translator):
    try:
        # Translate using the pipeline
        translation = translator(text, truncation=True)
        return translation[0]["translation_text"]  # Extract translated text
    except Exception as e:
        print(f"Error translating text: {text}\n{e}")
        return text  # Return original text if translation fails

# Load the dataset
df = pd.read_csv(input_file)

# Check if 'text' column exists
if "text" not in df.columns:
    raise ValueError("The CSV file does not contain a 'text' column.")

# Translate the 'text' column
df["text"] = df["text"].apply(lambda x: translate_text(x, translator))

# Save the updated dataframe to a new CSV
df.to_csv(output_file, index=False)

print(f"Translation completed. Translated file saved at: {output_file}")

Translation completed. Translated file saved at: /kaggle/working/rus-to-eng.csv


In [ ]:
df.to_csv('rus-to-eng.csv', index=False)